In [1]:
%load_ext autoreload
%autoreload 2

import drukarnia_api

Слава Україні 🇺🇦


### author

In [2]:
# create an author
author = drukarnia_api.Author(username='<your username>')

In [3]:
# Context manager creates an aiohttp session
async with author:
    # Password and email are not stored.
    await author.login(email='<your email>', password='<your password>')

In [4]:
async with author:
    follower, = await author.get_followers(n_collect=1)

    # collect profile's data about your follower
    await follower.collect_data()

    print('my follower\'s name is', await follower.name)
    print('my follower created account at', await follower.created_at)
    print('am I subscribed to my follower? -', await follower.is_subscribed)

my follower's name is Читач🍌
my follower created account at 2023-04-15 12:28:04.246000
am I subscribed to my follower? - False


In [5]:
async with follower:
    await follower.subscribe()

    # update data by requesting new one
    await follower.collect_data()

    print('am I subscribed to my follower now? -', await follower.is_subscribed)

am I subscribed to my follower now? - True


In [6]:
async with follower:
    fol1, = await follower.get_followings(n_collect=1)
    print('my follower also follows', await fol1.name)

my follower also follows ブラックライト


### Article

In [7]:
async with follower:
    article, *_ = await follower.articles
    print(f'{await follower.name} has an article named "{await article.title}"')

Читач🍌 has an article named "Кого я ще не читаю?"


In [9]:
async with article:
    await article.collect_data()

    print('Article\'s length is', await article.read_time, ' seconds')

Article's length is 48.0  seconds


In [11]:
async with article:
    print('I liked this article', await article.my_likes, ' times')
    print('this article has', await article.number_of_likes, ' likes')

    await article.like_article(1)
    await article.collect_data()

    print('I liked this article', await article.my_likes, ' times')
    print('this article has', await article.number_of_likes, ' likes')

I liked this article 10  times
this article has 620  likes
I liked this article 1  times
this article has 611  likes


In [12]:
async with article:
    print('This article has', await article.number_of_comments, ' comments')

    await article.post_comment('Hello, World!')
    await article.collect_data()

    print('Now this article has', await article.number_of_comments, ' comments')

This article has 54  comments
Now this article has 55  comments


### Comment

In [13]:
async with article:
    # ⚠️ONLY tree heads⚠️, replies are obtained from comment object itself
    comment, *_ = await article.comments

    print(f'article {await article.seo_title} has {len(_) + 1} comments')

article Кого я ще не читаю? has 48 comments


In [14]:
{
'text': await comment.text,
'number of likes': await comment.number_of_likes,
'number of replies': await comment.number_of_replies,
'have I liked it': await comment.is_liked,
'owner\'s name': await (await comment.owner).name
}

{'text': '<p>я в вашій чарівній колекції вже є, але тепер Я ПІДПИСУЮСЬ НА ВАС, ЩОБ МАТИ ДОСТУП ДО <strong>УСІХ</strong> АВТОРІВ НА ЦІЙ ПЛАТФОРМІ, МУХАХАХАХХАА! </p>',
 'number of likes': 18,
 'number of replies': 3,
 'have I liked it': False,
 "owner's name": 'сироманка🧀'}

In [15]:
async with comment:
    replies = await comment.get_replies()

print(f'comment has {len(replies)} replies')
[await repl.text for repl in replies]

comment has 3 replies


['<p><a data-type="mention" class="fw-semibold no-decor mention" target="_blank" data-id="cupomanka" href="/cupomanka">@cupomanka</a> а я підпишусь на вас</p>',
 '<p><a data-type="mention" class="fw-semibold no-decor mention" target="_blank" data-id="cupomanka" href="/cupomanka">@cupomanka</a> А в мене показує менше ніж 1,7к🥺</p>',
 '<p><a data-type="mention" class="fw-semibold no-decor mention" target="_blank" data-id="Result" href="/Result">@Result</a> та, поки що через обмеження показується лише останні 100 читачів/авторів, а не фул список. Сподіваємось пофіксять :’D</p>']

### Tag

In [16]:
async with article:
    tag, *_ = await article.article_tags

    print(f'article {await article.title} has {len(_) + 1} tags')

article Кого я ще не читаю? has 5 tags


In [18]:
print(f'tag {await tag.slug} has been mentioned {await tag.mentions_num} times')

tag ukrayina has been mentioned 439 times


In [19]:
async with tag:
    related = await tag.related_tags()

print(f'tag {await tag.slug} has {len(related)} related tags')
[await t.name for t in related]

tag ukrayina has 6 related tags


['Війна', 'Політика', 'Українізація', 'Історія', 'Культура', 'Література']

### How aiohttp sessions are managed?

#### Automatic Session Management:

> After creating an object:
> ```py
> author = drukarnia_api.Author(username='drukarnia')
> ```
> ##### Headers (drukarnia_api.Headers) and cookies (drukarnia_api.DrukarniaCookies) are instantly created. However, the session will be created/closed only inside the context manager (__aenter__() and __aexit__()):
> ```py
> async with author:
>    follower, *others = await author.get_followers(n_collect=1)
>    print(await follower.name)
> ```
> ##### All drukarnia_api objects (e.g., follower ~ drukarnia_api.Author) created inside the context manager will inherit the manager's (in this context, author) session and credentials. This means that inside the same context window, all created objects can make requests without any additional context managers, with the same authentication cookies. Once the context manager is exited, the session will be closed, but credentials will be saved. Example:
> ```py
> async with author:
>     follower, *others = await author.get_followers(n_collect=1)
>
>     # Subscribe to your follower from the author's account
>     await follower.subscribe()
>
> async with follower:
>     # Unsubscribe to your follower from the author's account
>     await follower.subscribe(unsubscribe=True)
> ```
> ##### Note that this logic is also applicable for properties.
> ##### Example | (credentials shared):

> ```py
> async with author:
>     article, *_ = await author.articles
>
> async with article:
>     await article.like_article(1)
> ```
> ##### Example || (credentials NOT shared):

> ```py
> article, *_ = await author.articles
>
> async with article:
>     # Error: 'User needs to sign up first'
>     await article.like_article(1)
> ```

#### Manual Session Management:

> ##### You can create your own session and pass it to the object. In this case, you don't need to use a context manager; however, it's your responsibility to close the session. Example:

> ```py
> import drukarnia_api
> import aiohttp
>
> custom_session = aiohttp.ClientSession(
>     base_url=drukarnia_api.Connection.base_url,
>     # cookies=drukarnia_api.DrukarniaCookies(),
>     # headers=...
> )
>
> author = drukarnia_api.Author(username='drukarnia', session=custom_session)
>
> # Follower's session == custom_session
> follower, *others = await author.get_followers(n_collect=1)
>
> await custom_session.close()
> ```

<img src="https://media.tenor.com/v88ccbQm9GQAAAAM/thank-you-for-your-time-charlie-hudson.gif" width="500" height="500" />